In [ ]:
from IPython.core.display import HTML
with open('../style.css') as f:
    css = f.read()
HTML(css)

In [ ]:
%load_ext nb_mypy

In [ ]:
from typing import Callable

In [ ]:
Row   = tuple[str, ...]
State = tuple[Row, ...]

# Connect Four

This notebook defines the game [Connect Four](https://en.wikipedia.org/wiki/Connect_Four).
Connect Four is played on a board of dimension $6 \times 7$, i.e. there are $6$ rows and $7$ columns.  Instead of `Red` and `Yellow` we call the players `X` and `O`.  Player `X` starts.  Player `X` and `O` take turns to choose columns that are not yet filled.  When player `X` chooses column `c`,  the first non-empty field in column `c` is filled with an `'X'`.  Likewise, when player `O` chooses column `c`,  the first non-empty field in column `c` is filled with an `'O'`.  Rows are numbered from the bottom up, i.e. the bottom row is row $0$.  The goal of the game for player `X` is to get four consecutive `'X's` into a row, column, or diagonal line, while player `O` needs to get four consecutive `'O's` into a row, column, or diagonal line.

In [ ]:
gPlayers = [ 'X', 'O' ]

States are represented as tuples of tuples.  The game starts with an empty board.  An empty field on the board is represented as the string `' '`.

In [ ]:
gStart: State = tuple( tuple(' ' for col in range(7)) for row in range(6))
gStart

In [ ]:
def toString(S: State) -> str:
    result = ''
    for row in S:
        line = '|'
        for element in row:
            if element == ' ':
                line += '⬜|'
            elif element == 'X':
                line += '🔴|'
            elif element == 'O':
                line += '🔵|'
        result = line + '\n' + result
    return result

In [ ]:
print(toString(gStart))

In [ ]:
R1 = ('X', 'X', 'O', 'X', 'X', 'O', ' ')
R2 = ('X', 'O', 'X', 'O', 'X', 'O', ' ')
R3 = (' ', ' ', 'O', 'X', 'X', 'X', ' ')
R4 = (' ', ' ', 'X', 'O', 'O', 'O', ' ')
R5 = (' ', ' ', 'O', 'O', 'X', ' ', ' ')
R6 = (' ', ' ', ' ', 'O', ' ', ' ', ' ')
gTestState = (R1, R2, R3, R4, R5, R6)
print(toString(gTestState))

The function `to_list` transforms a tuple of tuples into a list of lists.

In [ ]:
def to_list(S: State) -> list[list[str]]: 
    return [list(row) for row in S]

The function `to_tuple` transforms a list of lists into a tuple of tuples.

In [ ]:
def to_tuple(S: list[list[str]]) -> State: 
    return tuple(tuple(row) for row in S)

The function `find_empty` takes two arguments:
- `State` is a description of the board,
- `col`   specifies a column, i.e. it is an integer from the set $\{0, \cdots, 6\}$.

Given the `State` the function `find_empty(State, col)` returns the smallest $\texttt{row} \in \{0, \cdots, 5\}$ such that 

```
    State[row][col] == ' '
```
holds.  If the specified column is already completely filled, then instead `None` is returned.

In [ ]:
def find_empty(S: State, col: int) -> int | None:
    for row in range(5+1):
        if S[row][col] == ' ':
            return row
    return None

Given a `State` and the `player` who has the next move, the function `next_states(State, player)` computes the list of states that can be reached from `State`.

In [ ]:
def next_states(S: State, player: str) -> list[State]:
    Empty = [(row, col) for col in range(6+1) 
                        if ((row := find_empty(S, col)) != None)]
    Result = []
    for (row, col) in Empty: 
        NextState           = to_list(S)
        NextState[row][col] = player # type: ignore
        NextState           = to_tuple(NextState)
        Result.append(NextState)
    return Result

In [ ]:
for ns in next_states(gTestState, 'X'):
    print(toString(ns))

The variable `All_Lines` collects the coordinates of all groups of four fields that are consecutive horizontally, vertically, or diagonally.

In [ ]:
All_Lines: list[list[tuple[int, int]]]
# horizontal lines
All_Lines  = [ [ (row  , col+x) for x in range(3+1) ] for col in range(3+1)
                                                      for row in range(5+1) 
             ]
# vertical lines
All_Lines += [ [ (row+x, col  ) for x in range(3+1) ] for col in range(6+1)
                                                      for row in range(2+1) 
             ]
# rising diagonals
All_Lines += [ [ (row+x, col+x) for x in range(3+1) ] for col in range(3+1)
                                                      for row in range(2+1)
             ]
# falling diagonals
All_Lines += [ [ (row+x, col-x) for x in range(3+1) ] for col in range(3, 6+1)
                                                      for row in range(2+1)
             ]

In [ ]:
len(All_Lines)

Given a `State` the function `top_line_filled(State)` checks whether all marks in the top line of the given board are filled.

In [ ]:
def top_line_filled(S: State) -> bool:
    return ' ' not in S[5] 

The function `utility` takes one argument:
- `State` is a tuple of tuples representing the board.
 
The function returns `1` if player `X` has won the game, `-1` if player `O` has won the game, `0` if its a draw, and `None` if the game has not yet been decided.

In [ ]:
def utility(S: State) -> int | None:
    # all lines are checked whether they contain 4 identical nonempty marks 
    for Line in All_Lines:
        Chars = { S[r][c] for (r, c) in Line }
        if len(Chars) == 1 and Chars != { ' ' }:
            if Chars == { 'X' }:
                return  1
            else:
                return -1
    # no winner so far, check for a draw
    if top_line_filled(S):  # no empty squares
        return 0
    return None

The function heuristic tries to guess the value of a state.  Every three `X`'s in a line that can still be extended to four in a line are counted as `+1/10`, and every two `X`'s in a line that can be extended to four in a line are counted as `+1/100`.  Likewise, three `O`'s in a line count as `-1/10` and two `O`'s count as `-1/100`.

In [ ]:
def heuristic(S: State) -> float:
    result = 0.0
    # all lines are checked whether they contain 3 identical nonempty marks 
    for Line in All_Lines:
        List = [ S[r][c] for (r,c) in Line if  S[r][c] != ' ' ]
        if len(List) == 3:
            Chars = set(List)
            if len(Chars) == 1:
                if Chars == { 'X' }: 
                    result += 1/10
                else:
                    result -= 1/10
        if len(List) == 2:
            Chars = set(List)
            if len(Chars) == 1:
                if Chars == { 'X' }: 
                    result += 1/100
                else:
                    result -= 1/100
        # prefer middle row
    for row in range(5+1):
        if S[row][3] == 'X':
            result += 1/20
        elif S[row][3] == 'O':
            result -= 1/20
    return result

In [ ]:
heuristic(gTestState)

`finished(State)` is `True` if the game is over.

In [ ]:
def finished(S: State) -> bool:
    return utility(S) != None

The function `get_move` asks the user to input a move in the format `r,c` where `r` is the row and the `c` is the column where the next symbol is to be placed.

In [ ]:
def get_move(S: State) -> State:
    while True:
        col    = input("Enter column here: ")
        colInt = int(col)
        rowInt = find_empty(S, colInt)
        if rowInt != None:
            L = to_list(S)
            L[rowInt][colInt] = 'O' # type: ignore
            return to_tuple(L)
        else:
            print("Don't cheat.  Please try again.")           

This function informs the user about the result of the game once the game is finished.

In [ ]:
def final_msg(S: State) -> bool:
    if finished(S):
        if utility(S) == -1:
            print("You have won!")
        elif utility(S) == 1:
            print("You have lost!")
        else:
            print("It's a draw.");
        return True
    return False